In [446]:
import pandas as pd
import json
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import URL

In [447]:
pd.options.display.max_columns = 100

In [448]:
username = 'root'
password='root'
host='localhost'
database='jmdb2'

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")


In [449]:
df = pd.read_csv("emdb.csv",encoding="utf-16", sep = ";")

In [450]:
df = df.rename(columns = {v : v.strip() for v in df.columns})

In [451]:
print(df.columns)

Index(['Numéro', 'Titre', 'Titre alternatif', 'Année', 'Pays', 'Genre(s)',
       'Langues', 'Durée', 'Chemin de l'affiche', 'Réalisateur(s)',
       'Scénariste(s)', 'Compositeur(s)', 'Studio', 'Distribution',
       'Bande-annonce', 'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
       'Commentaires', 'Marqueurs', 'Support', 'Taille du fichier',
       'ID TheMovieDb', 'ID IMDb'],
      dtype='object')


# table movie , director et intermediaire

In [452]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [453]:
# Modifier le chemin de l'affiche pour ne conserver que la dernière partie
df_movie["Chemin de l'affiche"] = df_movie["Chemin de l'affiche"].astype(str).apply(lambda x: x.split('\\')[-1])

# Afficher le DataFrame résultant
print(df_movie.head())  # Affiche les premières lignes du DataFrame

   Numéro                              Titre  \
0    3874            The Sensualist (vostfr)   
1    3873  Les 4 Fantastiques : Premiers pas   
2    3872                            Sinners   
3    3871                      Simple mortel   
4    3870            Prison On Fire (vostfr)   

                                Titre alternatif Année  Durée  \
0                    井原西鶴　好色一代男 (Titre original)  1991     54   
1  The Fantastic 4: First Steps (Titre original)  2025    115   
2                                            NaN  2025    138   
3                                            NaN  1990     85   
4                          監獄風雲 (Titre original)  1987    101   

  Chemin de l'affiche                        Bande-annonce  \
0          003958.jpg                                  NaN   
1          003957.jpg  www.youtube.com/watch?v=d7zkZXNbzcs   
2          003956.jpg  www.youtube.com/watch?v=rOulkHoq2w8   
3          003955.jpg                                  NaN   
4         

In [454]:
list_director_movie = ["Réalisateur(s)", 'Numéro']
df_director_movie = df.copy()[list_director_movie]

In [455]:
# df_director_movie ["Réalisateur(s)"] = df_director_movie["Réalisateur(s)"].str.split(",")
df_director_movie["Réalisateur(s)"] = df_director_movie["Réalisateur(s)"].str.split(",\s*")

In [456]:
df_director_movie  = df_director_movie.explode("Réalisateur(s)")

In [457]:
df_director  = df_director_movie.explode("Réalisateur(s)").drop_duplicates("Réalisateur(s)").reset_index(drop=True).reset_index()
df_director = df_director.rename(columns={"index":"id"}).drop(columns="Numéro")

#.reset_index().rename(columns={"index":"id"})

In [458]:
df_intermediaire_movie_director = df_director.merge(df_director_movie, on = "Réalisateur(s)").drop(columns="Réalisateur(s)")

In [459]:
# Remplacer les valeurs NULL de la colonne 'id' par 0
df_director['id'] = df_director['id'].fillna(0)
# Convertir la colonne 'id' en type entier
df_director['id'] = df_director['id'].astype(int)

# Réorganiser les ID si nécessaire pour éviter les conflits
max_id = df_director['id'].max()
if max_id > 0:
    df_director['id'] = df_director['id'] + 1

In [460]:
display(df_director)

,id,Réalisateur(s)
0,1,Yukio Abe
1,2,Matt Shakman
2,3,Ryan Coogler
3,4,Pierre Jolivet
4,5,Ringo Lam Ling-Tung
...,...,...
1912,1913,Richard C. Sarafian
1913,1914,Chris Petit
1914,1915,Brett Ratner
1915,1916,Buster Keaton


In [461]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3874,The Sensualist (vostfr),井原西鶴 好色一代男 (Titre original),1991,54,003958.jpg,NaN,NaN,A story of a man's erotic experiences since th...,F:\#DIVX\DIVX MOVIEZ\ANIM\The Sensualist (1991...,AVI (Microsoft),NaN,Fichier multimédia,423 MB,movie/65206,2017720.0
1,3873,Les 4 Fantastiques : Premiers pas,The Fantastic 4: First Steps (Titre original),2025,115,003957.jpg,www.youtube.com/watch?v=d7zkZXNbzcs,Bienvenue dans la famille.,Dans le contexte vibrant d’un monde rétro-futu...,F:\#DIVX\DIVX MOVIEZ\MARVEL DC\The Fantastic F...,Matroska (MKV),NaN,Fichier multimédia,9.48 GB,movie/617126,10676052.0
2,3872,Sinners,NaN,2025,138,003956.jpg,www.youtube.com/watch?v=rOulkHoq2w8,Une nuit pour survivre.,Alors qu’ils cherchent à s’affranchir d’un lou...,F:\#DIVX\DIVX MOVIEZ\MadMov\Sinners (2025) MUL...,Matroska (MKV),NaN,Fichier multimédia,8.33 GB,movie/1233413,31193180.0
3,3871,Simple mortel,NaN,1990,85,003955.jpg,NaN,NaN,Un jeune spécialiste en langues anciennes reço...,F:\#DIVX\DIVX MOVIEZ\POLAR\Simple Mortel (1991...,Matroska (MKV),NaN,Fichier multimédia,2.09 GB,movie/45131,102930.0
4,3870,Prison On Fire (vostfr),監獄風雲 (Titre original),1987,101,003954.jpg,www.youtube.com/watch?v=qF8jHJO7_g8,NaN,Deux amis enfermés en prison vont devenir des ...,F:\#DIVX\DIVX MOVIEZ\POLAR\Prison on Fire -Rin...,Matroska (MKV),NaN,Fichier multimédia,1.94 GB,movie/53653,93304.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3869,5,le mécano de la général,The General (Titre original),1926,79,000005.jpg,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3870,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,000004.jpg,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3871,3,Les feux de la rampe,Limelight (Titre original),1952,137,000003.jpg,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3872,2,Les temps modernes,Modern Times (Titre original),1936,87,000002.jpg,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [462]:
display(df_intermediaire_movie_director)

,id,Numéro
0,0,3874
1,1,3873
2,2,3872
3,2,2879
4,3,3871
...,...,...
4182,1912,20
4183,1913,18
4184,1914,15
4185,1915,5


In [463]:
# Trouver l'index de la ligne avec le 'Réalisateur(s)' NULL
null_director_index = df_director[df_director['Réalisateur(s)'].isnull()].index

# Si une ligne avec 'Réalisateur(s)' NULL est trouvée
if not null_director_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_director.loc[null_director_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_director['id'].max()
    if max_id > 0:
        df_director.loc[df_director.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_director.head())

# Enregistrer le DataFrame modifié dans la table 'director' de la base de données
df_director.to_sql("director", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_director = df_director.merge(df_director_movie, on="Réalisateur(s)").drop(columns="Réalisateur(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_director.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_director.to_sql("movie_director", index=False, con=engine, if_exists="replace")

     id       Réalisateur(s)
0  1918            Yukio Abe
1     2         Matt Shakman
2     3         Ryan Coogler
3     4       Pierre Jolivet
4     5  Ringo Lam Ling-Tung
     id  Numéro
0  1918    3874
1     2    3873
2     3    3872
3     3    2879
4     4    3871


4187

In [464]:
# df_director.to_sql("director",index=False,con = engine,if_exists = "replace" )

In [465]:
# df_intermediaire_movie_director.to_sql("movie_director",index=False,con = engine,if_exists = "replace" )

In [466]:
df_movie.to_sql("movies",index=False,con = engine,if_exists = "replace" )

3874

# table movie, screenwriter et intermediaire

In [467]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [468]:
list_screenwriter_movie = ["Scénariste(s)", 'Numéro']
df_screenwriter_movie = df.copy()[list_screenwriter_movie]

In [469]:
df_screenwriter_movie ["Scénariste(s)"] = df_screenwriter_movie["Scénariste(s)"].str.split(",\s*")

In [470]:
df_screenwriter_movie  = df_screenwriter_movie.explode("Scénariste(s)")

In [471]:
df_screenwriter  = df_screenwriter_movie.explode("Scénariste(s)").drop_duplicates("Scénariste(s)").reset_index(drop=True).reset_index()
df_screenwriter = df_screenwriter.rename(columns={"index":"id"}).drop(columns="Numéro")

In [472]:
df_intermediaire_movie_screenwriter = df_screenwriter.merge(df_screenwriter_movie, on = "Scénariste(s)").drop(columns="Scénariste(s)")

In [473]:
display(df_screenwriter)

,id,Scénariste(s)
0,0,Saikaku Ihara
1,1,Eiichi Yamamoto
2,2,Josh Friedman
3,3,Eric Pearson
4,4,Jeff Kaplan
...,...,...
4127,4127,Ted Tally
4128,4128,David Veloz
4129,4129,Richard Rutowski
4130,4130,Buster Keaton


In [474]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3874,The Sensualist (vostfr),井原西鶴 好色一代男 (Titre original),1991,54,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A story of a man's erotic experiences since th...,F:\#DIVX\DIVX MOVIEZ\ANIM\The Sensualist (1991...,AVI (Microsoft),NaN,Fichier multimédia,423 MB,movie/65206,2017720.0
1,3873,Les 4 Fantastiques : Premiers pas,The Fantastic 4: First Steps (Titre original),2025,115,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=d7zkZXNbzcs,Bienvenue dans la famille.,Dans le contexte vibrant d’un monde rétro-futu...,F:\#DIVX\DIVX MOVIEZ\MARVEL DC\The Fantastic F...,Matroska (MKV),NaN,Fichier multimédia,9.48 GB,movie/617126,10676052.0
2,3872,Sinners,NaN,2025,138,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=rOulkHoq2w8,Une nuit pour survivre.,Alors qu’ils cherchent à s’affranchir d’un lou...,F:\#DIVX\DIVX MOVIEZ\MadMov\Sinners (2025) MUL...,Matroska (MKV),NaN,Fichier multimédia,8.33 GB,movie/1233413,31193180.0
3,3871,Simple mortel,NaN,1990,85,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Un jeune spécialiste en langues anciennes reço...,F:\#DIVX\DIVX MOVIEZ\POLAR\Simple Mortel (1991...,Matroska (MKV),NaN,Fichier multimédia,2.09 GB,movie/45131,102930.0
4,3870,Prison On Fire (vostfr),監獄風雲 (Titre original),1987,101,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=qF8jHJO7_g8,NaN,Deux amis enfermés en prison vont devenir des ...,F:\#DIVX\DIVX MOVIEZ\POLAR\Prison on Fire -Rin...,Matroska (MKV),NaN,Fichier multimédia,1.94 GB,movie/53653,93304.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3869,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3870,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3871,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3872,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [475]:
display(df_intermediaire_movie_screenwriter)

,id,Numéro
0,0,3874
1,0,1009
2,1,3874
3,1,2878
4,2,3873
...,...,...
7480,4127,15
7481,4128,14
7482,4129,14
7483,4130,5


In [476]:
# Trouver l'index de la ligne avec le 'Scénariste(s)' NULL
null_screenwriter_index = df_screenwriter[df_screenwriter['Scénariste(s)'].isnull()].index

# Si une ligne avec 'Scénariste(s)' NULL est trouvée
if not null_screenwriter_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_screenwriter.loc[null_screenwriter_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_screenwriter['id'].max()
    if max_id > 0:
        df_screenwriter.loc[df_screenwriter.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_screenwriter.head())

# Enregistrer le DataFrame modifié dans la table 'screenwriter' de la base de données
df_screenwriter.to_sql("screenwriter", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_screenwriter = df_screenwriter.merge(df_screenwriter_movie, on="Scénariste(s)").drop(columns="Scénariste(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_screenwriter.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_screenwriter.to_sql("movie_screenwriter", index=False, con=engine, if_exists="replace")


     id    Scénariste(s)
0  4132    Saikaku Ihara
1     1  Eiichi Yamamoto
2     2    Josh Friedman
3     3     Eric Pearson
4     4      Jeff Kaplan
     id  Numéro
0  4132    3874
1  4132    1009
2     1    3874
3     1    2878
4     2    3873


7485

In [477]:
# df_screenwriter.to_sql("screenwriter",index=False,con = engine,if_exists = "replace" )

In [478]:
# df_intermediaire_movie_screenwriter.to_sql("movie_screenwriter",index=False,con = engine,if_exists = "replace" )

# table movie, music et intermediaire

In [479]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [480]:
list_music_movie = ["Compositeur(s)", 'Numéro']
df_music_movie = df.copy()[list_music_movie]

In [481]:
df_music_movie ["Compositeur(s)"] = df_music_movie["Compositeur(s)"].str.split(",\s*")

In [482]:
df_music_movie  = df_music_movie.explode("Compositeur(s)")

In [483]:
df_music  = df_music_movie.explode("Compositeur(s)").drop_duplicates("Compositeur(s)").reset_index(drop=True).reset_index()
df_music = df_music.rename(columns={"index":"id"}).drop(columns="Numéro")

In [484]:
df_intermediaire_movie_music = df_music.merge(df_music_movie, on = "Compositeur(s)").drop(columns="Compositeur(s)")

In [485]:
display(df_music)

,id,Compositeur(s)
0,0,NaN
1,1,Michael Giacchino
2,2,Ludwig Göransson
3,3,Serge Perathoner
4,4,Jannick Top
...,...,...
1371,1371,Machine Head
1372,1372,Photek
1373,1373,John Frizzell
1374,1374,Leonard Cohen


In [486]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3874,The Sensualist (vostfr),井原西鶴 好色一代男 (Titre original),1991,54,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A story of a man's erotic experiences since th...,F:\#DIVX\DIVX MOVIEZ\ANIM\The Sensualist (1991...,AVI (Microsoft),NaN,Fichier multimédia,423 MB,movie/65206,2017720.0
1,3873,Les 4 Fantastiques : Premiers pas,The Fantastic 4: First Steps (Titre original),2025,115,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=d7zkZXNbzcs,Bienvenue dans la famille.,Dans le contexte vibrant d’un monde rétro-futu...,F:\#DIVX\DIVX MOVIEZ\MARVEL DC\The Fantastic F...,Matroska (MKV),NaN,Fichier multimédia,9.48 GB,movie/617126,10676052.0
2,3872,Sinners,NaN,2025,138,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=rOulkHoq2w8,Une nuit pour survivre.,Alors qu’ils cherchent à s’affranchir d’un lou...,F:\#DIVX\DIVX MOVIEZ\MadMov\Sinners (2025) MUL...,Matroska (MKV),NaN,Fichier multimédia,8.33 GB,movie/1233413,31193180.0
3,3871,Simple mortel,NaN,1990,85,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Un jeune spécialiste en langues anciennes reço...,F:\#DIVX\DIVX MOVIEZ\POLAR\Simple Mortel (1991...,Matroska (MKV),NaN,Fichier multimédia,2.09 GB,movie/45131,102930.0
4,3870,Prison On Fire (vostfr),監獄風雲 (Titre original),1987,101,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=qF8jHJO7_g8,NaN,Deux amis enfermés en prison vont devenir des ...,F:\#DIVX\DIVX MOVIEZ\POLAR\Prison on Fire -Rin...,Matroska (MKV),NaN,Fichier multimédia,1.94 GB,movie/53653,93304.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3869,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3870,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3871,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3872,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [487]:
display(df_intermediaire_movie_music)

,id,Numéro
0,0,3874
1,0,3864
2,0,3861
3,0,3855
4,0,3850
...,...,...
4304,1371,46
4305,1372,46
4306,1373,42
4307,1374,14


In [488]:
# Trouver l'index de la ligne avec le 'Compositeur(s)' NULL
null_music_index = df_music[df_music['Compositeur(s)'].isnull()].index

# Si une ligne avec 'Compositeur(s)' NULL est trouvée
if not null_music_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_music.loc[null_music_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_music['id'].max()
    if max_id > 0:
        df_music.loc[df_music.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_music.head())

# Enregistrer le DataFrame modifié dans la table 'music' de la base de données
df_music.to_sql("music", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_music = df_music.merge(df_music_movie, on="Compositeur(s)").drop(columns="Compositeur(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_music.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_music.to_sql("movie_music", index=False, con=engine, if_exists="replace")


     id     Compositeur(s)
0  1376                NaN
1     1  Michael Giacchino
2     2   Ludwig Göransson
3     3   Serge Perathoner
4     4        Jannick Top
     id  Numéro
0  1376    3874
1  1376    3864
2  1376    3861
3  1376    3855
4  1376    3850


4309

In [489]:
# df_music.to_sql("music",index=False,con = engine,if_exists = "replace" )

In [490]:
df_intermediaire_movie_music.to_sql("movie_music",index=False,con = engine,if_exists = "replace" )

4309

# table movie, studio et intermediaire

In [491]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [492]:
list_studio_movie = ["Studio", 'Numéro']
df_studio_movie = df.copy()[list_studio_movie]

In [493]:
df_studio_movie ["Studio"] = df_studio_movie["Studio"].str.split(",\s*")

In [494]:
df_studio_movie  = df_studio_movie.explode("Studio")

In [495]:
df_studio  = df_studio_movie.explode("Studio").drop_duplicates("Studio").reset_index(drop=True).reset_index()
df_studio = df_studio.rename(columns={"index":"id"}).drop(columns="Numéro")

In [496]:
df_intermediaire_movie_studio = df_studio.merge(df_studio_movie, on = "Studio").drop(columns="Studio")

In [497]:
display(df_studio)

,id,Studio
0,0,NaN
1,1,Marvel Studios
2,2,Warner Bros. Pictures
3,3,Cinema City Company Limited
4,4,Paramount Pictures
...,...,...
1047,1047,ADR Distribution
1048,1048,A-Film Distribution
1049,1049,Dybex
1050,1050,British Empire Films Australia


In [498]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3874,The Sensualist (vostfr),井原西鶴 好色一代男 (Titre original),1991,54,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A story of a man's erotic experiences since th...,F:\#DIVX\DIVX MOVIEZ\ANIM\The Sensualist (1991...,AVI (Microsoft),NaN,Fichier multimédia,423 MB,movie/65206,2017720.0
1,3873,Les 4 Fantastiques : Premiers pas,The Fantastic 4: First Steps (Titre original),2025,115,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=d7zkZXNbzcs,Bienvenue dans la famille.,Dans le contexte vibrant d’un monde rétro-futu...,F:\#DIVX\DIVX MOVIEZ\MARVEL DC\The Fantastic F...,Matroska (MKV),NaN,Fichier multimédia,9.48 GB,movie/617126,10676052.0
2,3872,Sinners,NaN,2025,138,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=rOulkHoq2w8,Une nuit pour survivre.,Alors qu’ils cherchent à s’affranchir d’un lou...,F:\#DIVX\DIVX MOVIEZ\MadMov\Sinners (2025) MUL...,Matroska (MKV),NaN,Fichier multimédia,8.33 GB,movie/1233413,31193180.0
3,3871,Simple mortel,NaN,1990,85,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Un jeune spécialiste en langues anciennes reço...,F:\#DIVX\DIVX MOVIEZ\POLAR\Simple Mortel (1991...,Matroska (MKV),NaN,Fichier multimédia,2.09 GB,movie/45131,102930.0
4,3870,Prison On Fire (vostfr),監獄風雲 (Titre original),1987,101,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=qF8jHJO7_g8,NaN,Deux amis enfermés en prison vont devenir des ...,F:\#DIVX\DIVX MOVIEZ\POLAR\Prison on Fire -Rin...,Matroska (MKV),NaN,Fichier multimédia,1.94 GB,movie/53653,93304.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3869,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3870,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3871,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3872,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [499]:
display(df_intermediaire_movie_studio)

,id,Numéro
0,0,3874
1,0,3871
2,0,3831
3,0,3828
4,0,3820
...,...,...
3872,1047,182
3873,1048,172
3874,1049,169
3875,1050,108


In [500]:
# Trouver l'index de la ligne avec le 'Studio' NULL
null_studio_index = df_studio[df_studio['Studio'].isnull()].index

# Si une ligne avec 'Studio' NULL est trouvée
if not null_studio_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_studio.loc[null_studio_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_studio['id'].max()
    if max_id > 0:
        df_studio.loc[df_studio.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_studio.head())

# Enregistrer le DataFrame modifié dans la table 'studio' de la base de données
df_studio.to_sql("studio", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_studio = df_studio.merge(df_studio_movie, on="Studio").drop(columns="Studio")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_studio.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_studio.to_sql("movie_studio", index=False, con=engine, if_exists="replace")


     id                       Studio
0  1052                          NaN
1     1               Marvel Studios
2     2        Warner Bros. Pictures
3     3  Cinema City Company Limited
4     4           Paramount Pictures
     id  Numéro
0  1052    3874
1  1052    3871
2  1052    3831
3  1052    3828
4  1052    3820


3877

In [501]:
# df_studio.to_sql("studio",index=False,con = engine,if_exists = "replace" )

In [502]:
df_intermediaire_movie_studio.to_sql("movie_studio",index=False,con = engine,if_exists = "replace" )

3877

# table movie, casting & intermediaire

In [503]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [504]:
list_casting_movie = ["Distribution", 'Numéro']
df_casting_movie = df.copy()[list_casting_movie]

In [505]:
# Nettoyer les noms d'acteurs en supprimant "(voix)" et autres parenthèses similaires
df_casting_movie["Distribution"] = df_casting_movie["Distribution"].str.replace(r"\s*\(.*?\)", "", regex=True)

In [506]:
df_casting_movie ["Distribution"] = df_casting_movie["Distribution"].str.split(",\s*")

In [507]:
df_casting_movie  = df_casting_movie.explode("Distribution")

In [508]:
df_casting  = df_casting_movie.explode("Distribution").drop_duplicates("Distribution").reset_index(drop=True).reset_index()
df_casting = df_casting.rename(columns={"index":"id"}).drop(columns="Numéro")

In [509]:
df_intermediaire_movie_casting = df_casting.merge(df_casting_movie, on = "Distribution").drop(columns="Distribution")

In [510]:
display(df_casting)

,id,Distribution
0,0,Masao Imanishi
1,1,Minori Matsushima
2,2,Ichiro Nagai
3,3,Taeko Nakanishi
4,4,Osamu Saka
...,...,...
10141,10141,Paulette Goddard
10142,10142,Henry Bergman
10143,10143,Tiny Sandford
10144,10144,Jackie Coogan


In [511]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3874,The Sensualist (vostfr),井原西鶴 好色一代男 (Titre original),1991,54,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A story of a man's erotic experiences since th...,F:\#DIVX\DIVX MOVIEZ\ANIM\The Sensualist (1991...,AVI (Microsoft),NaN,Fichier multimédia,423 MB,movie/65206,2017720.0
1,3873,Les 4 Fantastiques : Premiers pas,The Fantastic 4: First Steps (Titre original),2025,115,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=d7zkZXNbzcs,Bienvenue dans la famille.,Dans le contexte vibrant d’un monde rétro-futu...,F:\#DIVX\DIVX MOVIEZ\MARVEL DC\The Fantastic F...,Matroska (MKV),NaN,Fichier multimédia,9.48 GB,movie/617126,10676052.0
2,3872,Sinners,NaN,2025,138,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=rOulkHoq2w8,Une nuit pour survivre.,Alors qu’ils cherchent à s’affranchir d’un lou...,F:\#DIVX\DIVX MOVIEZ\MadMov\Sinners (2025) MUL...,Matroska (MKV),NaN,Fichier multimédia,8.33 GB,movie/1233413,31193180.0
3,3871,Simple mortel,NaN,1990,85,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Un jeune spécialiste en langues anciennes reço...,F:\#DIVX\DIVX MOVIEZ\POLAR\Simple Mortel (1991...,Matroska (MKV),NaN,Fichier multimédia,2.09 GB,movie/45131,102930.0
4,3870,Prison On Fire (vostfr),監獄風雲 (Titre original),1987,101,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=qF8jHJO7_g8,NaN,Deux amis enfermés en prison vont devenir des ...,F:\#DIVX\DIVX MOVIEZ\POLAR\Prison on Fire -Rin...,Matroska (MKV),NaN,Fichier multimédia,1.94 GB,movie/53653,93304.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3869,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3870,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3871,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3872,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [512]:
display(df_intermediaire_movie_casting)

,id,Numéro
0,0,3874
1,1,3874
2,2,3874
3,3,3874
4,4,3874
...,...,...
18383,10141,2
18384,10142,2
18385,10143,2
18386,10144,1


In [513]:
# Trouver l'index de la ligne avec la 'Distribution' NULL
null_casting_index = df_casting[df_casting['Distribution'].isnull()].index

# Si une ligne avec 'Distribution' NULL est trouvée
if not null_casting_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_casting.loc[null_casting_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_casting['id'].max()
    if max_id > 0:
        df_casting.loc[df_casting.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_casting.head())

# Enregistrer le DataFrame modifié dans la table 'casting' de la base de données
df_casting.to_sql("casting", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_casting = df_casting.merge(df_casting_movie, on="Distribution").drop(columns="Distribution")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_casting.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_casting.to_sql("movie_casting", index=False, con=engine, if_exists="replace")


      id       Distribution
0  10146     Masao Imanishi
1      1  Minori Matsushima
2      2       Ichiro Nagai
3      3    Taeko Nakanishi
4      4         Osamu Saka
      id  Numéro
0  10146    3874
1      1    3874
2      2    3874
3      3    3874
4      4    3874


18388

In [514]:
# df_casting.to_sql("casting",index=False,con = engine,if_exists = "replace" )

In [515]:
# df_intermediaire_movie_casting.to_sql("movie_casting",index=False,con = engine,if_exists = "replace" )

# table movie, country & intermediaire

In [516]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [517]:
list_country_movie = ["Pays", 'Numéro']
df_country_movie = df.copy()[list_country_movie]

In [518]:
df_country_movie ["Pays"] = df_country_movie["Pays"].str.split(",\s*")

In [519]:
df_country_movie  = df_country_movie.explode("Pays")

In [520]:
df_country  = df_country_movie.explode("Pays").drop_duplicates("Pays").reset_index(drop=True).reset_index()
df_country = df_country.rename(columns={"index":"id"}).drop(columns="Numéro")

In [521]:
df_intermediaire_movie_country = df_country.merge(df_country_movie, on = "Pays").drop(columns="Pays")

In [522]:
display(df_country)

,id,Pays
0,0,Japon
1,1,Etats-Unis
2,2,France
3,3,Hong Kong
4,4,Suède
5,5,Italie
6,6,Autriche
7,7,Allemagne
8,8,Royaume-Uni
9,9,Russie


In [523]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3874,The Sensualist (vostfr),井原西鶴 好色一代男 (Titre original),1991,54,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A story of a man's erotic experiences since th...,F:\#DIVX\DIVX MOVIEZ\ANIM\The Sensualist (1991...,AVI (Microsoft),NaN,Fichier multimédia,423 MB,movie/65206,2017720.0
1,3873,Les 4 Fantastiques : Premiers pas,The Fantastic 4: First Steps (Titre original),2025,115,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=d7zkZXNbzcs,Bienvenue dans la famille.,Dans le contexte vibrant d’un monde rétro-futu...,F:\#DIVX\DIVX MOVIEZ\MARVEL DC\The Fantastic F...,Matroska (MKV),NaN,Fichier multimédia,9.48 GB,movie/617126,10676052.0
2,3872,Sinners,NaN,2025,138,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=rOulkHoq2w8,Une nuit pour survivre.,Alors qu’ils cherchent à s’affranchir d’un lou...,F:\#DIVX\DIVX MOVIEZ\MadMov\Sinners (2025) MUL...,Matroska (MKV),NaN,Fichier multimédia,8.33 GB,movie/1233413,31193180.0
3,3871,Simple mortel,NaN,1990,85,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Un jeune spécialiste en langues anciennes reço...,F:\#DIVX\DIVX MOVIEZ\POLAR\Simple Mortel (1991...,Matroska (MKV),NaN,Fichier multimédia,2.09 GB,movie/45131,102930.0
4,3870,Prison On Fire (vostfr),監獄風雲 (Titre original),1987,101,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=qF8jHJO7_g8,NaN,Deux amis enfermés en prison vont devenir des ...,F:\#DIVX\DIVX MOVIEZ\POLAR\Prison on Fire -Rin...,Matroska (MKV),NaN,Fichier multimédia,1.94 GB,movie/53653,93304.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3869,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3870,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3871,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3872,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [524]:
display(df_intermediaire_movie_country)

,id,Numéro
0,0,3874
1,0,3867
2,0,3854
3,0,3841
4,0,3840
...,...,...
3869,44,853
3870,45,921
3871,46,769
3872,47,631


In [525]:
# Trouver l'index de la ligne avec le 'Pays' NULL
null_country_index = df_country[df_country['Pays'].isnull()].index

# Si une ligne avec 'Pays' NULL est trouvée
if not null_country_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_country.loc[null_country_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_country['id'].max()
    if max_id > 0:
        df_country.loc[df_country.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_country.head())

# Enregistrer le DataFrame modifié dans la table 'country' de la base de données
df_country.to_sql("country", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_country = df_country.merge(df_country_movie, on="Pays").drop(columns="Pays")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_country.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_country.to_sql("movie_country", index=False, con=engine, if_exists="replace")

   id        Pays
0  48       Japon
1   1  Etats-Unis
2   2      France
3   3   Hong Kong
4   4       Suède
   id  Numéro
0  48    3874
1  48    3867
2  48    3854
3  48    3841
4  48    3840


3874

In [526]:
# df_country.to_sql("country",index=False,con = engine,if_exists = "replace" )

In [527]:
df_intermediaire_movie_country.to_sql("movie_country",index=False,con = engine,if_exists = "replace" )

3874

# table movie, genre & intermediaire

In [528]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [529]:
list_genre_movie = ["Genre(s)", 'Numéro']
df_genre_movie = df.copy()[list_genre_movie]

In [530]:
df_genre_movie ["Genre(s)"] = df_genre_movie["Genre(s)"].str.split(",\s*")

In [531]:
df_genre_movie  = df_genre_movie.explode("Genre(s)")

In [532]:
df_genre  = df_genre_movie.explode("Genre(s)").drop_duplicates("Genre(s)").reset_index(drop=True).reset_index()
df_genre = df_genre.rename(columns={"index":"id"}).drop(columns="Numéro")

In [533]:
df_intermediaire_movie_genre = df_genre.merge(df_genre_movie, on = "Genre(s)").drop(columns="Genre(s)")

In [534]:
display(df_genre)

,id,Genre(s)
0,0,Animation
1,1,Adulte
2,2,Science-fiction
3,3,Super-Héros
4,4,Horreur
5,5,Action
6,6,Suspense
7,7,Drame
8,8,Policier
9,9,Fantastique


In [535]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3874,The Sensualist (vostfr),井原西鶴 好色一代男 (Titre original),1991,54,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A story of a man's erotic experiences since th...,F:\#DIVX\DIVX MOVIEZ\ANIM\The Sensualist (1991...,AVI (Microsoft),NaN,Fichier multimédia,423 MB,movie/65206,2017720.0
1,3873,Les 4 Fantastiques : Premiers pas,The Fantastic 4: First Steps (Titre original),2025,115,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=d7zkZXNbzcs,Bienvenue dans la famille.,Dans le contexte vibrant d’un monde rétro-futu...,F:\#DIVX\DIVX MOVIEZ\MARVEL DC\The Fantastic F...,Matroska (MKV),NaN,Fichier multimédia,9.48 GB,movie/617126,10676052.0
2,3872,Sinners,NaN,2025,138,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=rOulkHoq2w8,Une nuit pour survivre.,Alors qu’ils cherchent à s’affranchir d’un lou...,F:\#DIVX\DIVX MOVIEZ\MadMov\Sinners (2025) MUL...,Matroska (MKV),NaN,Fichier multimédia,8.33 GB,movie/1233413,31193180.0
3,3871,Simple mortel,NaN,1990,85,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Un jeune spécialiste en langues anciennes reço...,F:\#DIVX\DIVX MOVIEZ\POLAR\Simple Mortel (1991...,Matroska (MKV),NaN,Fichier multimédia,2.09 GB,movie/45131,102930.0
4,3870,Prison On Fire (vostfr),監獄風雲 (Titre original),1987,101,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=qF8jHJO7_g8,NaN,Deux amis enfermés en prison vont devenir des ...,F:\#DIVX\DIVX MOVIEZ\POLAR\Prison on Fire -Rin...,Matroska (MKV),NaN,Fichier multimédia,1.94 GB,movie/53653,93304.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3869,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3870,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3871,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3872,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [536]:
display(df_intermediaire_movie_genre)

,id,Numéro
0,0,3874
1,0,3863
2,0,3855
3,0,3847
4,0,3831
...,...,...
8699,37,36
8700,37,35
8701,38,3113
8702,38,389


In [537]:
# Trouver l'index de la ligne avec le 'Genre(s)' NULL
null_genre_index = df_genre[df_genre['Genre(s)'].isnull()].index

# Si une ligne avec 'Genre(s)' NULL est trouvée
if not null_genre_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_genre.loc[null_genre_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_genre['id'].max()
    if max_id > 0:
        df_genre.loc[df_genre.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_genre.head())

# Enregistrer le DataFrame modifié dans la table 'genre' de la base de données
df_genre.to_sql("genre", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_genre = df_genre.merge(df_genre_movie, on="Genre(s)").drop(columns="Genre(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_genre.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_genre.to_sql("movie_genre", index=False, con=engine, if_exists="replace")


   id         Genre(s)
0  38        Animation
1   1           Adulte
2   2  Science-fiction
3   3      Super-Héros
4   4          Horreur
   id  Numéro
0  38    3874
1  38    3863
2  38    3855
3  38    3847
4  38    3831


8704

In [538]:
df_genre.to_sql("genre",index=False,con = engine,if_exists = "replace" )

39

In [539]:
df_intermediaire_movie_genre.to_sql("movie_genre",index=False,con = engine,if_exists = "replace" )

8704

# table movie, language & intermediaire

In [540]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [541]:
list_language_movie = ["Langues", 'Numéro']
df_language_movie = df.copy()[list_language_movie]

In [542]:
df_language_movie ["Langues"] = df_language_movie["Langues"].str.split(",\s*")

In [543]:
df_language_movie  = df_language_movie.explode("Langues")

In [544]:
df_language  = df_language_movie.explode("Langues").drop_duplicates("Langues").reset_index(drop=True).reset_index()
df_language = df_language.rename(columns={"index":"id"}).drop(columns="Numéro")

In [545]:
df_intermediaire_movie_language = df_language.merge(df_language_movie, on = "Langues").drop(columns="Langues")

In [546]:
display(df_language)

,id,Langues
0,0,Japonais
1,1,Anglais
2,2,Français
3,3,Cantonais
4,4,Italien
5,5,Suédois
6,6,NaN
7,7,Russe
8,8,Allemand
9,9,Vietnamien


In [547]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3874,The Sensualist (vostfr),井原西鶴 好色一代男 (Titre original),1991,54,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A story of a man's erotic experiences since th...,F:\#DIVX\DIVX MOVIEZ\ANIM\The Sensualist (1991...,AVI (Microsoft),NaN,Fichier multimédia,423 MB,movie/65206,2017720.0
1,3873,Les 4 Fantastiques : Premiers pas,The Fantastic 4: First Steps (Titre original),2025,115,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=d7zkZXNbzcs,Bienvenue dans la famille.,Dans le contexte vibrant d’un monde rétro-futu...,F:\#DIVX\DIVX MOVIEZ\MARVEL DC\The Fantastic F...,Matroska (MKV),NaN,Fichier multimédia,9.48 GB,movie/617126,10676052.0
2,3872,Sinners,NaN,2025,138,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=rOulkHoq2w8,Une nuit pour survivre.,Alors qu’ils cherchent à s’affranchir d’un lou...,F:\#DIVX\DIVX MOVIEZ\MadMov\Sinners (2025) MUL...,Matroska (MKV),NaN,Fichier multimédia,8.33 GB,movie/1233413,31193180.0
3,3871,Simple mortel,NaN,1990,85,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Un jeune spécialiste en langues anciennes reço...,F:\#DIVX\DIVX MOVIEZ\POLAR\Simple Mortel (1991...,Matroska (MKV),NaN,Fichier multimédia,2.09 GB,movie/45131,102930.0
4,3870,Prison On Fire (vostfr),監獄風雲 (Titre original),1987,101,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=qF8jHJO7_g8,NaN,Deux amis enfermés en prison vont devenir des ...,F:\#DIVX\DIVX MOVIEZ\POLAR\Prison on Fire -Rin...,Matroska (MKV),NaN,Fichier multimédia,1.94 GB,movie/53653,93304.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3869,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3870,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3871,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3872,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [548]:
display(df_intermediaire_movie_language)

,id,Numéro
0,0,3874
1,0,3867
2,0,3854
3,0,3841
4,0,3840
...,...,...
5431,42,1060
5432,43,1053
5433,43,930
5434,43,926


In [549]:
# Trouver l'index de la ligne avec les 'Langues' NULL
null_language_index = df_language[df_language['Langues'].isnull()].index

# Si une ligne avec 'Langues' NULL est trouvée
if not null_language_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_language.loc[null_language_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_language['id'].max()
    if max_id > 0:
        df_language.loc[df_language.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_language.head())

# Enregistrer le DataFrame modifié dans la table 'language' de la base de données
df_language.to_sql("language", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_language = df_language.merge(df_language_movie, on="Langues").drop(columns="Langues")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_language.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_language.to_sql("movie_language", index=False, con=engine, if_exists="replace")


   id    Langues
0  45   Japonais
1   1    Anglais
2   2   Français
3   3  Cantonais
4   4    Italien
   id  Numéro
0  45    3874
1  45    3867
2  45    3854
3  45    3841
4  45    3840


5436

In [550]:
# df_language.to_sql("language",index=False,con = engine,if_exists = "replace" )

In [551]:
df_intermediaire_movie_language.to_sql("movie_language",index=False,con = engine,if_exists = "replace" )

5436

# table, movie, tag & intermediaire

In [552]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [553]:
list_tag_movie = ["Etiquettes", 'Numéro']
df_tag_movie = df.copy()[list_tag_movie]

KeyError: "['Etiquettes'] not in index"

In [ ]:
df_tag_movie ["Etiquettes"] = df_tag_movie["Etiquettes"].str.split(",\s*")

In [ ]:
df_tag_movie  = df_tag_movie.explode("Etiquettes")

In [ ]:
df_tag = df_tag_movie.explode("Etiquettes").drop_duplicates("Etiquettes").reset_index(drop=True).reset_index()
df_tag = df_tag.rename(columns={"index":"id"}).drop(columns="Numéro")

In [ ]:
df_intermediaire_movie_tag = df_tag.merge(df_tag_movie, on = "Etiquettes").drop(columns="Etiquettes")

In [ ]:
display(df_tag)

In [ ]:
display(df_movie)

In [ ]:
display(df_intermediaire_movie_tag)

In [ ]:
# Trouver l'index de la ligne avec les 'Etiquettes' NULL
null_tag_index = df_tag[df_tag['Etiquettes'].isnull()].index

# Si une ligne avec 'Etiquettes' NULL est trouvée
if not null_tag_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_tag.loc[null_tag_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_tag['id'].max()
    if max_id > 0:
        df_tag.loc[df_tag.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_tag.head())

# Enregistrer le DataFrame modifié dans la table 'tag' de la base de données
df_tag.to_sql("tag", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_tag = df_tag.merge(df_tag_movie, on="Etiquettes").drop(columns="Etiquettes")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_tag.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_tag.to_sql("movie_tag", index=False, con=engine, if_exists="replace")


In [ ]:
# df_tag.to_sql("tag",index=False,con = engine,if_exists = "replace" )

In [ ]:
# df_intermediaire_movie_tag.to_sql("movie_tag",index=False,con = engine,if_exists = "replace" )